In [1]:
import os
import cv2
import numpy as np
from rembg import remove
import random

In [2]:
def add_text_centered(img, text, font, font_scale, color, thickness, bg_color, offset_y=0):
    """
    Adds centered text to an image with a background rectangle.
    
    Args:
        img: The image to draw on.
        text: The text to add.
        font: The font to use.
        font_scale: The scale of the font.
        color: The text color (BGR).
        thickness: The thickness of the text.
        bg_color: The background color (BGR) for the text.
        offset_y: Vertical offset for the text position.
    
    Returns:
        The image with the centered text.
    """
    text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
    text_width, text_height = text_size
    img_h, img_w = img.shape[:2]

    # Calculate the position to center the text
    x = (img_w - text_width) // 2
    y = (img_h + text_height) // 2 + offset_y

    # Draw background rectangle
    padding = 10
    cv2.rectangle(
        img,
        (x - padding, y - text_height - padding),
        (x + text_width + padding, y + padding),
        bg_color,
        cv2.FILLED,
    )

    # Add the text on top of the rectangle
    cv2.putText(img, text, (x, y), font, font_scale, color, thickness, cv2.LINE_AA)
    return img


In [3]:
def add_image(base_img, img_to_add, pos):
    """Adds an image to the base image at a specific position."""
    x, y = pos
    h, w = img_to_add.shape[:2]
    base_h, base_w = base_img.shape[:2]
    
    # Ensure the image fits within the base image
    if y + h > base_h or x + w > base_w:
        h = min(h, base_h - y)
        w = min(w, base_w - x)
        img_to_add = cv2.resize(img_to_add, (w, h))
    
    # Add the image with transparency if available
    for c in range(3):  # Assuming RGB
        base_img[y:y+h, x:x+w, c] = (
            img_to_add[:, :, c] * (img_to_add[:, :, 3] / 255.0) +
            base_img[y:y+h, x:x+w, c] * (1.0 - img_to_add[:, :, 3] / 255.0)
        )
    return base_img

In [4]:
def remove_background(image_path):
    """Removes the background from an image."""
    with open(image_path, "rb") as f:
        img_bytes = f.read()
    try:
        result = remove(img_bytes)
        return cv2.imdecode(np.frombuffer(result, np.uint8), cv2.IMREAD_UNCHANGED)
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

In [5]:
# def generate_random_positions(base_img, img_shapes, padding=10, max_attempts=50000):
#     """
#     Generates random positions for images to be placed on the base image.
#     """
#     base_h, base_w = base_img.shape[:2]
#     resized_shapes = []
#     positions = []

#     for img_shape in img_shapes:
#         h, w = img_shape[:2]

#         # Resize if the image exceeds the base size
#         if h > base_h or w > base_w:
#             scale = min(base_h / h, base_w / w) * 0.2  # Scale proportionally with margin
#             h, w = int(h * scale), int(w * scale)

#         resized_shapes.append((h, w))

#         # Find a valid position
#         attempts = 0
#         while attempts < max_attempts:
#             x = random.randint(0, base_w - w) if base_w - w > 0 else 0
#             y = random.randint(0, base_h - h) if base_h - h > 0 else 0

#             # Check for overlap
#             if not any(
#                 (abs(x - px) < w + padding and abs(y - py) < h + padding)
#                 for px, py in positions
#             ):
#                 positions.append((x, y))
#                 break
#             attempts += 1

#         # If no space is found, resize the image and retry
#         if attempts == max_attempts:
#             print(f"Warning: Could not place image {img_shape}. Reducing size.")
#             scale = 0.5
#             h, w = int(h * scale), int(w * scale)
#             resized_shapes[-1] = (h, w)
#             attempts = 0

#     return positions, resized_shapes

def generate_random_positions(base_img, img_shapes, padding=10, max_attempts=50000):
    """
    Generates random positions for images to be placed on the base image.
    """
    base_h, base_w = base_img.shape[:2]
    resized_shapes = []
    positions = []

    for img_shape in img_shapes:
        h, w = img_shape[:2]

        # Resize if the image exceeds the base size
        if h > base_h or w > base_w:
            scale = min(base_h / h, base_w / w) * 0.2  # Scale proportionally with margin
            h, w = int(h * scale), int(w * scale)

        resized_shapes.append((h, w))

        # Find a valid position
        attempts = 0
        while attempts < max_attempts:
            x = random.randint(0, base_w - w) if base_w - w > 0 else 0
            y = random.randint(0, base_h - h) if base_h - h > 0 else 0

            # Check for overlap
            if not any(
                (abs(x - px) < w + padding and abs(y - py) < h + padding)
                for px, py in positions
            ):
                positions.append((x, y))
                break
            attempts += 1

        # If no space is found, resize the image and retry
        if attempts == max_attempts:
            print(f"Warning: Could not place image {img_shape}. Reducing size.")
            scale = 0.5
            h, w = int(h * scale), int(w * scale)
            resized_shapes[-1] = (h, w)
            attempts = 0

    return positions, resized_shapes

In [6]:
base_img = cv2.imread("base_image.png")
base_img = cv2.resize(base_img, (1280, 720)) 

In [7]:
texto = input('Ingrese el título para el collage: ')

In [8]:
name_channel = input('Ingrese el nombre del canal: ')

In [9]:
# Process images from folder
carpeta = "/mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/6_create_portada/imagenes"
archivos = os.listdir(carpeta)
image_paths = [os.path.join(carpeta, archivo) for archivo in archivos if os.path.isfile(os.path.join(carpeta, archivo))]


In [10]:
images = []
for path in image_paths:
    img = remove_background(path)
    if img is not None:
        images.append(img)

if not images:
    raise ValueError("No se pudieron procesar imágenes válidas. Verifica la carpeta de entrada.")


In [11]:
# Generate random positions
positions, resized_shapes = generate_random_positions(base_img, [img.shape for img in images])


In [12]:
# Resize images
images_resized = [
    cv2.resize(img, (w, h)) for img, (h, w) in zip(images, resized_shapes)
]


In [13]:
# Add images to base image
for img, pos in zip(images_resized, positions):
    base_img = add_image(base_img, img, pos)

In [14]:
# Add centered text over the images
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
text_color = (255, 255, 255)  # White
bg_color = (0, 0, 0)  # Black
thickness = 2

In [15]:
base_img = add_text_centered(base_img, texto, font, font_scale, text_color, thickness, bg_color, offset_y=-30)
base_img = add_text_centered(base_img, name_channel, font, font_scale, text_color, thickness, bg_color, offset_y=30)


In [16]:
# Save final thumbnail
cv2.imwrite("thumbnail.jpg", base_img)
print("Miniatura creada y guardada como 'thumbnail.jpg'.")

Miniatura creada y guardada como 'thumbnail.jpg'.
